In [2]:
import os
from mysql.connector import connect, Error as MySQLError
from neo4j import GraphDatabase, basic_auth, exceptions as Neo4jExceptions
from openai import OpenAI, APIError

# --- 配置 (保持不变) ---
MYSQL_CONFIG = {
    'host': '127.0.0.1',
    'port': 3306,
    'user': 'root',
    'password': '112233445566',
    'database': 'wikidata_rdbms'
}
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "112233445566"
DEEPSEEK_API_KEY = os.getenv("DeepSeek_API_KEY")
DEEPSEEK_BASE_URL = "https://api.deepseek.com"
DEEPSEEK_MODEL = "deepseek-chat"

# --- 查询定义 (修改问题 1 和 2 的 SQL/Cypher) ---
queries = [
    {
        "question": "中华人民共和国的首都是什么？",
        "sql": """
            SELECT target_entity.label AS capital_name
            FROM Relationships r
            JOIN Entities target_entity ON r.target_uri = target_entity.uri
            WHERE r.source_uri = 'http://www.wikidata.org/entity/Q148' -- 使用中国 URI 查询
              AND r.relationship_type = 'hasCapital';
        """,
        "cypher": """
            MATCH (country:Country {uri: 'http://www.wikidata.org/entity/Q148'})-[:HAS_CAPITAL]->(capital:City) -- 使用中国 URI 查询
            RETURN capital.name AS capital_name
        """
    },
    {
        "question": "德国的人口是多少？",
        "sql": """
            SELECT cd.population
            FROM CountryDetails cd
            WHERE cd.country_uri = 'http://www.wikidata.org/entity/Q183'; -- 使用德国 URI 查询
        """,
        "cypher": """
            MATCH (country:Country {uri: 'http://www.wikidata.org/entity/Q183'}) -- 使用德国 URI 查询
            RETURN country.population AS population
        """
    },
    {
        "question": "列出面积大于 5,000,000 平方千米的国家及其首都。",
        "sql": """
            SELECT
                country_entity.label AS country_name,
                capital_entity.label AS capital_name
            FROM CountryDetails cd
            JOIN Entities country_entity ON cd.country_uri = country_entity.uri
            LEFT JOIN Relationships r ON cd.country_uri = r.source_uri AND r.relationship_type = 'hasCapital'
            LEFT JOIN Entities capital_entity ON r.target_uri = capital_entity.uri
            WHERE cd.area > 5000000
            ORDER BY country_entity.label;
        """,
        "cypher": """
            MATCH (country:Country)
            WHERE country.area > 5000000
            OPTIONAL MATCH (country)-[:HAS_CAPITAL]->(capital:City)
            RETURN country.name AS country_name, capital.name AS capital_name
            ORDER BY country_name
        """
    }
]

# --- 数据库和 API 查询函数 (保持不变) ---
def execute_sql_query(query):
    """执行 SQL 查询并返回结果"""
    results = []
    try:
        with connect(**MYSQL_CONFIG) as connection:
            with connection.cursor(dictionary=True) as cursor:
                cursor.execute(query)
                results = cursor.fetchall()
                if not results:
                    # 检查是否有行被找到但结果是 NULL
                    if cursor.rowcount == 0:
                        return "查询成功，但未找到匹配的记录。"
                    else: # 找到了记录，但选择的列可能是 NULL
                        # 对于聚合函数或特定列查询，fetchall 可能返回 [{col: None}]
                        # 检查结果是否真的为空或只包含 None
                        all_none = True
                        if results:
                            for row in results:
                                if any(val is not None for val in row.values()):
                                    all_none = False
                                    break
                        if all_none:
                            return "查询成功，找到记录但请求的数据为空 (NULL)。"
                        else:
                            return results # 返回包含非 None 值的结果
                return results
    except MySQLError as e:
        return f"MySQL Error: {e}"
    except Exception as e:
        return f"An unexpected error occurred during SQL query: {e}"

def execute_cypher_query(query):
    """执行 Cypher 查询并返回结果"""
    results = []
    driver = None
    try:
        driver = GraphDatabase.driver(NEO4J_URI, auth=basic_auth(NEO4J_USER, NEO4J_PASSWORD))
        with driver.session(database="neo4j") as session:
            result_cursor = session.run(query)
            results = [record.data() for record in result_cursor]
            summary = result_cursor.consume() # 获取查询摘要信息

            if not results:
                # 检查是否有节点或关系被匹配，但返回的属性是 null
                # summary.counters.nodes_matched > 0 or summary.counters.relationships_matched > 0
                # 简单的处理：如果没有结果行，就认为没找到
                return "查询成功，但未返回结果。"

            # 检查返回的结果是否都为 null
            all_null = True
            if results:
                for record in results:
                    if any(val is not None for val in record.values()):
                        all_null = False
                        break
            if all_null:
                return "查询成功，找到匹配但请求的数据为空 (null)。"

            return results
    except Neo4jExceptions.ServiceUnavailable:
        return f"Neo4j Error: 无法连接到 {NEO4J_URI}。请确保 Neo4j 服务器正在运行。"
    except Neo4jExceptions.AuthError:
        return f"Neo4j Error: 认证失败，请检查用户名或密码。"
    except Exception as e:
        return f"An unexpected error occurred during Cypher query: {e}"
    finally:
        if driver:
            driver.close()

def ask_deepseek(question):
    """使用 DeepSeek API 提问并返回答案"""
    if not DEEPSEEK_API_KEY:
        return "DeepSeek Error: API Key 未在环境变量 DeepSeek_API_KEY 中设置。"
    try:
        client = OpenAI(api_key=DEEPSEEK_API_KEY, base_url=DEEPSEEK_BASE_URL)
        response = client.chat.completions.create(
            model=DEEPSEEK_MODEL,
            messages=[
                {"role": "system", "content": "你是一个乐于助人的助手。"},
                {"role": "user", "content": question}
            ],
            stream=False,
            max_tokens=500,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except APIError as e:
        return f"DeepSeek API Error: {e}"
    except Exception as e:
        return f"An unexpected error occurred while calling DeepSeek API: {e}"

# --- 主执行逻辑 (保持不变) ---
if __name__ == "__main__":
    for i, q in enumerate(queries):
        print(f"\n--- 查询 {i+1} ---")
        print(f"问题: {q['question']}")
        print("-" * 20)

        # 1. 执行 SQL 查询
        print("MySQL (SQL) 查询结果:")
        sql_result = execute_sql_query(q['sql'])
        if isinstance(sql_result, list):
            for row in sql_result:
                print(row)
        else:
            print(sql_result)
        print("-" * 20)

        # 2. 执行 Cypher 查询
        print("Neo4j (Cypher) 查询结果:")
        cypher_result = execute_cypher_query(q['cypher'])
        if isinstance(cypher_result, list):
            for record in cypher_result:
                print(record)
        else:
            print(cypher_result)
        print("-" * 20)

        # 3. 执行 DeepSeek LLM 查询
        print("DeepSeek LLM 回答:")
        llm_answer = ask_deepseek(q['question'])
        print(llm_answer)
        print("-" * 20)

    print("\n--- 所有查询执行完毕 ---")


--- 查询 1 ---
问题: 中华人民共和国的首都是什么？
--------------------
MySQL (SQL) 查询结果:
{'capital_name': '北京市'}
--------------------
Neo4j (Cypher) 查询结果:
An unexpected error occurred during Cypher query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '使用中国': expected '(' (line 2, column 116 (offset: 116))
"            MATCH (country:Country {uri: 'http://www.wikidata.org/entity/Q148'})-[:HAS_CAPITAL]->(capital:City) -- 使用中国 URI 查询"
                                                                                                                    ^}
--------------------
DeepSeek LLM 回答:
中华人民共和国的首都是北京。北京是中国的政治、文化、国际交往和科技创新中心，承载着悠久的历史和丰富的文化遗产，同时也是现代化国际大都市。
--------------------

--- 查询 2 ---
问题: 德国的人口是多少？
--------------------
MySQL (SQL) 查询结果:
{'population': 84358845}
--------------------
Neo4j (Cypher) 查询结果:
An unexpected error occurred during Cypher query: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input '使用德国': expected '(' (line 2, column 85 (offset: 85))
" 